In [1]:
import os
import time
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import copy
import logging

from xml.dom.minidom import Document

In [2]:
input_path = "../data/negative_labeled.txt"
output_path = "../data/negative_labeled.xml"

In [3]:
class XmlMaker:
    def __init__(self,txtpath,xmlpath):
        self.txtPath = txtpath
        self.xmlPath = xmlpath
        
        
    def readtxt(self):
        txtfile = open(self.txtPath,"r",encoding='utf-8',errors='ignore')
        lines = txtfile.readlines()
                
        flag = 0
        self.review_list = []
        self.term_list = []
        self.aspect_list = []
        
        review_temp = ""
        term_temp = []
        aspect_temp = []
        
        cnt = 0
        for line in lines:
            if "EOF" in line:
                self.aspect_list.append(aspect_temp)
                break
            if "*****O" in line:
                flag = 1
                if cnt != 0:
                    self.aspect_list.append(aspect_temp)
                    aspect_temp = []
                cnt += 1
                continue
            elif "***T" in line:
                flag = 2
                self.review_list.append(review_temp)
                review_temp = ""
                continue
            elif "***A" in line:
                if flag == 1:
                    self.review_list.append(review_temp)
                    review_temp = ""
                    
                flag = 3
                self.term_list.append(term_temp)
                term_temp = []
                continue
                
            if flag == 1:
                review_temp = line.replace('\n','')
                continue
            if flag == 2:
                term_temp.append(line.replace('\n',''))
                continue
            if flag == 3:
                aspect_temp.append(line.replace('\n',''))
                continue

                
    def makexml(self):
        doc = Document()
        sentences = doc.createElement("sentences")
        objecname = "sentence"
        
        for i in range(len(self.review_list)):
            
            objectE = doc.createElement(objecname)
            objectE.setAttribute("id",str(i))
            
            text = doc.createElement("text")
            content = doc.createTextNode(self.review_list[i])
            text.appendChild(content)
            objectE.appendChild(text)
            
            if len(self.term_list[i]):
                j = 0
                aspectTerms = doc.createElement("aspectTerms")
                while j < len(self.term_list[i]):
                    aspectTerm = doc.createElement("aspectTerm")
                    aspectTerm.setAttribute("term",self.term_list[i][j])
                    aspectTerm.setAttribute("polarity",self.term_list[i][j+1])
                    aspectTerms.appendChild(aspectTerm)
                    j += 2
                objectE.appendChild(aspectTerms)
            
            j = 0
            aspectCategories = doc.createElement("aspectCategories")
            while j < len(self.aspect_list[i]):
                aspectCategory= doc.createElement("aspectCategory")
                aspectCategory.setAttribute("category",self.aspect_list[i][j])
                aspectCategory.setAttribute("attribute",self.aspect_list[i][j+1])
                aspectCategory.setAttribute("polarity",self.aspect_list[i][j+2])
                aspectCategories.appendChild(aspectCategory)
                j += 3
            objectE.appendChild(aspectCategories)
            
            sentences.appendChild(objectE)
            
        doc.appendChild(sentences)

        f = open(self.xmlPath, 'w')
        doc.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
        f.close()



In [4]:
read =XmlMaker(input_path,output_path)
read.readtxt()
# print(len(read.review_list),len(read.term_list),len(read.aspect_list))
read.makexml()